In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Real-time Fraud Detection

## Description of the use case
This use case involves detecting fraudulent transactions in real-time by using Redis Bloom Filters and Streams. By analyzing transaction data, businesses can identify and prevent fraud quickly.

## Redis Data Structures Used
- **Bloom Filters**: To check for known fraudulent entities.
- **Streams**: To capture and process real-time transaction data.
- **Hashes**: To store transaction details.


In [5]:
r.flushdb()
# Mock data for the use case
transactions = {
    'tx_1001': {'user_id': 'user_1', 'amount': 500, 'status': 'pending'},
    'tx_1002': {'user_id': 'user_2', 'amount': 3000, 'status': 'pending'},
    'tx_1003': {'user_id': 'user_3', 'amount': 700, 'status': 'pending'}
}

# Use Hashes to store transaction details
for tx_id, tx_info in transactions.items():
    r.hset(tx_id, mapping=tx_info)

# Initialize Bloom Filter for known fraudulent ips
r.execute_command('BF.RESERVE', 'fraudulent_ips', 0.01, 1000)
fraudulent_ips = ['193.176.23.7','212.18.104.18','213.55.85.202','27.254.235.3','34.66.72.251','45.148.10.240','45.148.10.79',
                  '51.254.101.166','59.12.160.91','80.242.208.68','88.214.48.16','88.214.48.17','88.214.48.18','91.205.219.185']
for ip in fraudulent_ips:
    r.bf().add('fraudulent_ips', ip)

# Use Streams to capture real-time transaction data
r.xadd('transactions', {'tx_id': 'tx_1001', 'user_id': 'user_1', 'amount': 500, 'ip': '10.2.4.255'})
r.xadd('transactions', {'tx_id': 'tx_1002', 'user_id': 'user_2', 'amount': 3000,'ip': '88.214.48.17'})
r.xadd('transactions', {'tx_id': 'tx_1003', 'user_id': 'user_3', 'amount': 3000, 'ip': '178.3.27.192'})

# Function to process transactions and detect fraud
def process_transactions():
    stream_entries = r.xrange('transactions')
    for entry in stream_entries:
        tx_id = entry[1]['tx_id']
        user_id = entry[1]['user_id']
        amount = int(entry[1]['amount'])
        ip = entry[1]['ip']
        if not r.bf().exists('fraudulent_ips', ip):
            r.hset(tx_id, 'status', 'approved')
            print(f"Transaction {tx_id} is approved.")
        else:
            r.hset(tx_id, 'status', 'fraudulent')
            print(f"Transaction {tx_id} is marked as fraudulent.") 

# Process the transactions
process_transactions()

# Print the updated transaction details
for tx_id in transactions.keys():
    print(f"{tx_id}: {r.hgetall(tx_id)}")


Transaction tx_1001 is approved.
Transaction tx_1002 is marked as fraudulent.
Transaction tx_1003 is approved.
tx_1001: {'user_id': 'user_1', 'amount': '500', 'status': 'approved'}
tx_1002: {'user_id': 'user_2', 'amount': '3000', 'status': 'fraudulent'}
tx_1003: {'user_id': 'user_3', 'amount': '700', 'status': 'approved'}
